In [ ]:
# Configure AWS profile for local development
%env AWS_PROFILE=platform-developer

# Imports and Neptune client initialization
# All dependencies managed by uv and pyproject.toml
import pprint
from typing import Any

import pandas as pd
from queries.concept_queries import CONCEPT_QUERY
from tabulate import tabulate

# Import catalogue graph modules (should work if running in proper uv environment)
from utils.aws import get_neptune_client

# Initialize Neptune client for local use
neptune_client = get_neptune_client(is_local=True)


# Utility functions for querying and result handling
def run_query(
    query: str, parameters: dict[str, Any] = None, label: str = None
) -> list[dict]:
    """Execute an openCypher query with optional parameters and timing."""
    if label:
        results = neptune_client.time_open_cypher_query(query, parameters or {}, label)
    else:
        results = neptune_client.run_open_cypher_query(query, parameters)
    return results


def display_results(results: list[dict], limit: int = 10) -> None:
    """Display query results in a formatted table."""
    if not results:
        print("No results to display")
        return

    df = pd.DataFrame(results)
    print(f"Results: {len(results)} rows, {len(df.columns)} columns")

    display_df = df.head(limit)
    print(tabulate(display_df, headers="keys", tablefmt="grid", showindex=False))

    if len(results) > limit:
        print(f"... and {len(results) - limit} more rows")


def display_raw_results(results: list[dict], limit: int = 3) -> None:
    """Display query results using pretty print for exploration."""
    if not results:
        print("No results to display")
        return

    print(
        f"🔍 Raw results structure (showing {min(limit, len(results))} of {len(results)}):"
    )
    for i, result in enumerate(results[:limit]):
        print(f"\n--- Result {i + 1} ---")
        pprint.pprint(result, width=80, depth=2)


def export_results_to_csv(results: list[dict], filename: str):
    """Export query results to a CSV file."""
    if results:
        pd.DataFrame(results).to_csv(filename, index=False)
        print(f"✓ Results exported to {filename}")

env: AWS_PROFILE=platform-developer
✅ Setup complete - ready to query the catalogue graph
✅ Setup complete - ready to query the catalogue graph


# Catalogue Graph Query Notebook

This notebook provides an interface for querying the Wellcome Collection catalogue graph using openCypher queries. It presumes you are running in the `uv` environment and are using VS Code with the appropriate Python kernel.

## Prerequisites
- Ensure you're running this notebook in the proper `uv` environment
- Have the `platform-developer` AWS profile configured
- Access to the Wellcome Collection VPN/network

## Quick Start
1. Make sure you're in the `catalogue_graph` directory
2. Run `uv sync` to ensure all dependencies are installed
3. Select the appropriate kernel `./.venv/bin/python` in VS Code
4. Run the setup cell above to initialize the connection 
5. Use the pre-built query functions or execute custom queries below

---

## Basic Graph Exploration

Let's start with some basic queries to understand the structure of the catalogue graph.

In [48]:
# Explore SourceConcepts by source
source_concepts_query = """
MATCH (sc:SourceConcept)
RETURN sc.source as source, count(sc) as count
ORDER BY count DESC
"""

results = run_query(source_concepts_query, label="Source concepts by source")
display_results(results)

Ran 'Source concepts by source' query in 3 seconds, retrieving 3 records.
Results: 3 rows, 2 columns
+-------------+---------+
| source      |   count |
+=============+=========+
| lc-subjects |  422417 |
+-------------+---------+
| wikidata    |   99822 |
+-------------+---------+
| nlm-mesh    |   30554 |
+-------------+---------+


In [50]:
# A query to get all relationship types in the graph
relationship_types_query = """
MATCH ()-[r]->()
RETURN DISTINCT type(r) as relationship_type, count(r) as count
ORDER BY count DESC
"""
results = run_query(relationship_types_query, label="Relationship types in the graph")
display_results(results)

Ran 'Relationship types in the graph' query in 48 seconds, retrieving 8 records.
Results: 8 rows, 2 columns
+---------------------+---------+
| relationship_type   |   count |
+=====================+=========+
| HAS_CONCEPT         | 3699553 |
+---------------------+---------+
| SAME_AS             | 3444092 |
+---------------------+---------+
| HAS_PARENT          | 2179670 |
+---------------------+---------+
| NARROWER_THAN       |  626785 |
+---------------------+---------+
| HAS_FIELD_OF_WORK   |  427145 |
+---------------------+---------+
| HAS_SOURCE_CONCEPT  |  297868 |
+---------------------+---------+
| HAS_PATH_IDENTIFIER |  275112 |
+---------------------+---------+
| RELATED_TO          |  238187 |
+---------------------+---------+


## Concept Queries

Query and explore concepts in the catalogue graph.

In [49]:
# Use the pre-built concept query to get a sample of concepts
# This query gets concepts with their linked source concepts
concept_sample_params = {
    "start_offset": 0,
    "limit": 3,  # Reduce to 3 for cleaner output with pretty print
}

print("Running pre-built concept query...")
print(f"Query parameters: {concept_sample_params}")
print(CONCEPT_QUERY)

results = run_query(CONCEPT_QUERY, concept_sample_params, "Sample concepts")

display_results(results)

results

Running pre-built concept query...
Query parameters: {'start_offset': 0, 'limit': 3}

    MATCH (concept:Concept)
    WITH concept ORDER BY concept.id
    SKIP $start_offset LIMIT $limit
    OPTIONAL MATCH (concept)-[:HAS_SOURCE_CONCEPT]->(linked_source_concept)-[:SAME_AS*0..]->(source_concept)
    OPTIONAL MATCH (source_concept)<-[:HAS_SOURCE_CONCEPT]-(same_as_concept)
    OPTIONAL MATCH (work)-[has_concept:HAS_CONCEPT]-(concept)
    RETURN 
        concept,
        collect(DISTINCT linked_source_concept) AS linked_source_concepts,
        collect(DISTINCT source_concept) AS source_concepts,
        collect(DISTINCT same_as_concept.id) AS same_as_concept_ids,
        collect(DISTINCT has_concept.referenced_type) AS concept_types        
    
Ran 'Sample concepts' query in 6 seconds, retrieving 3 records.
Results: 3 rows, 5 columns
+-----------------------------------------------------------------------------------------------------------------------------------------------------------

[{'concept': {'~id': 'a2233f9d',
   '~entityType': 'node',
   '~labels': ['Concept'],
   '~properties': {'id': 'a2233f9d',
    'label': 'Circle of State Librarians Conference 1979 : Kew, England)',
    'source': 'label-derived',
    'type': 'Meeting'}},
  'linked_source_concepts': [],
  'source_concepts': [],
  'same_as_concept_ids': [],
  'concept_types': ['Meeting']},
 {'concept': {'~id': 'a223f5a6',
   '~entityType': 'node',
   '~labels': ['Concept'],
   '~properties': {'id': 'a223f5a6',
    'label': 'Nelson, Geoffrey B. (Geoffrey Brian)',
    'source': 'lc-names',
    'type': 'Person'}},
  'linked_source_concepts': [{'~id': 'no2001062332',
    '~entityType': 'node',
    '~labels': ['SourceName'],
    '~properties': {'id': 'no2001062332',
     'label': 'Nelson, Geoffrey B. (Geoffrey Brian)',
     'source': 'lc-names'}}],
  'source_concepts': [{'~id': 'no2001062332',
    '~entityType': 'node',
    '~labels': ['SourceName'],
    '~properties': {'id': 'no2001062332',
     'label': 'Nel

In [44]:
# Search for concepts by label (case-insensitive)
def search_concepts(search_term: str, limit: int = 10):
    """Search for concepts containing the given term in their label."""
    search_query = """
    MATCH (c:Concept)
    WHERE toLower(c.label) CONTAINS toLower($search_term)
    RETURN c.id as concept_id, c.label as label
    ORDER BY c.label
    LIMIT $limit
    """

    params = {"search_term": search_term, "limit": limit}
    return run_query(search_query, params, f"Search for '{search_term}'")


# Example: Search for concepts related to "medicine"
search_term = "medicine"
search_results = search_concepts(search_term)
display_results(search_results)

Ran 'Search for 'medicine'' query in 6 seconds, retrieving 10 records.
Results: 10 rows, 2 columns
+--------------+----------------------------------------------------------------------------------------------------+
| concept_id   | label                                                                                              |
+==============+====================================================================================================+
| s2fgad3g     | ACM Conference on the History of Medical Informatics (1987 : National Library of Medicine)         |
+--------------+----------------------------------------------------------------------------------------------------+
| fsmfgzj2     | ATPM/Fogarty Symposium on Preventive and Community Medicine in Primary Care (1973 : Bethesda, Md.) |
+--------------+----------------------------------------------------------------------------------------------------+
| rt46hwa7     | Aboriginal Australians - Medicine                         

## Work Queries

Query and explore works in the catalogue graph and their relationships to concepts.

In [45]:
# Get sample works with their associated concepts
works_query = """
MATCH (w:Work)-[hc:HAS_CONCEPT]->(c:Concept)
WITH w, collect({concept: c, type: hc.referenced_type}) as concepts
RETURN w.id as work_id, 
       w.title as title,
       size(concepts) as concept_count,
       concepts[0..3] as sample_concepts
ORDER BY concept_count DESC
LIMIT 10
"""

works_results = run_query(works_query, label="Works with concepts")
display_results(works_results)

Ran 'Works with concepts' query in 67 seconds, retrieving 10 records.
Results: 10 rows, 4 columns
+-----------+---------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| work_id   | title   |   concept_count | sample_concepts                                                                                                                                       

## Custom Queries

Use this section to run your own openCypher queries.

In [46]:
# Template for custom queries
# Replace this with your own openCypher query

custom_query = """
MATCH (n)
RETURN count(n) as total_nodes
"""

# Optional parameters for your query
custom_params = {}

# Execute the query
custom_results = run_query(custom_query, custom_params, "Custom query")
display_results(custom_results)

Ran 'Custom query' query in 34 seconds, retrieving 1 records.
Results: 1 rows, 1 columns
+---------------+
|   total_nodes |
+===============+
|   1.61268e+07 |
+---------------+
